In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from pydantic.v1 import BaseModel  # Correcting the import for pydantic's BaseModel
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from langchain_experimental.tabular_synthetic_data.openai import (
    create_openai_data_generator,
    OPENAI_TEMPLATE,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_SUFFIX,
    SYNTHETIC_FEW_SHOT_PREFIX,
)

# Load environment variables from a .env file
load_dotenv()

# OpenAI API
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
class ProductListing(BaseModel):
    product_id: str
    category: SyntaxError
    brand: str
    condition: str
    price: float
    seller_reputation: int
    description: str
    suspected_fraud: bool

    class Config:
        arbitrary_types_allowed = True

In [3]:
examples = [
    # Electronics
    {
        "example": 'product_id: 1, category: Electronics, brand: Sony, condition: New, price: 950, seller_reputation: 4, description: "Brand new Sony Xperia 1", suspected_fraud: false'
    },
    {
        "example": 'product_id: 2, category: Electronics, brand: Xiaomi, condition: Used, price: 300, seller_reputation: 2, description: "Used Xiaomi Mi 9", suspected_fraud: false'
    },
    {
        "example": 'product_id: 3, category: Electronics, brand: OnePlus, condition: New, price: 700, seller_reputation: 5, description: "Brand new OnePlus 9 Pro", suspected_fraud: false'
    },
    {
        "example": 'product_id: 4, category: Electronics, brand: Huawei, condition: Used, price: 400, seller_reputation: 3, description: "Used Huawei P30", suspected_fraud: false'
    },
    {
        "example": 'product_id: 5, category: Electronics, brand: Google, condition: New, price: 850, seller_reputation: 5, description: "Brand new Google Pixel 5", suspected_fraud: false'
    },
    {
        "example": 'product_id: 6, category: Electronics, brand: Motorola, condition: Used, price: 250, seller_reputation: 4, description: "Used Motorola Edge", suspected_fraud: false'
    },
    {
        "example": 'product_id: 7, category: Electronics, brand: Oppo, condition: New, price: 600, seller_reputation: 4, description: "Brand new Oppo Find X3", suspected_fraud: false'
    },
    # Clothing
    {
        "example": 'product_id: 8, category: Clothing, brand: Puma, condition: New, price: 60, seller_reputation: 3, description: "Brand new Puma hoodie", suspected_fraud: false'
    },
    {
        "example": 'product_id: 9, category: Clothing, brand: Under Armour, condition: Used, price: 25, seller_reputation: 2, description: "Used Under Armour running shoes", suspected_fraud: false'
    },
    {
        "example": 'product_id: 10, category: Clothing, brand: Gucci, condition: New, price: 200, seller_reputation: 3, description: "Brand new Gucci handbag", suspected_fraud: True'
    },
    {
        "example": 'product_id: 11, category: Clothing, brand: Reebok, condition: Used, price: 35, seller_reputation: 3, description: "Used Reebok workout gear", suspected_fraud: false'
    },
    {
        "example": "product_id: 12, category: Clothing, brand: Levi's, condition: New, price: 45, seller_reputation: 4, description: \"Brand new Levi's jeans\", suspected_fraud: false"
    },
    {
        "example": 'product_id: 13, category: Clothing, brand: Zara, condition: Used, price: 20, seller_reputation: 2, description: "Used Zara summer dress", suspected_fraud: false'
    },
    {
        "example": 'product_id: 14, category: Clothing, brand: Hugo Boss, condition: New, price: 150, seller_reputation: 3, description: "Brand new Hugo Boss blazer", suspected_fraud: True'
    },
    {
        "example": 'product_id: 15, category: Clothing, brand: Uniqlo, condition: Used, price: 15, seller_reputation: 1, description: "Used Uniqlo T-shirt", suspected_fraud: false'
    },
    # Furniture
    {
        "example": 'product_id: 16, category: Furniture, brand: Burrow, condition: New, price: 1200, seller_reputation: 5, description: "Brand new Burrow modular sofa", suspected_fraud: false'
    },
    {
        "example": 'product_id: 17, category: Furniture, brand: FLOYD, condition: Used, price: 800, seller_reputation: 4, description: "Lightly used FLOYD sectional sofa", suspected_fraud: false'
    },
    {
        "example": 'product_id: 18, category: Furniture, brand: Maiden Home, condition: New, price: 2000, seller_reputation: 5, description: "Luxury Maiden Home leather armchair", suspected_fraud: false'
    },
    {
        "example": 'product_id: 19, category: Furniture, brand: Castlety, condition: New, price: 1500, seller_reputation: 3, description: "Brand new Castlety mid-century dining table", suspected_fraud: true'
    },
    {
        "example": 'product_id: 20, category: Furniture, brand: Benchmade Modern, condition: Used, price: 1800, seller_reputation: 4, description: "Used but excellent condition Benchmade Modern sofa", suspected_fraud: false'
    },
    {
        "example": 'product_id: 21, category: Furniture, brand: Burrow, condition: New, price: 1700, seller_reputation: 5, description: "Brand new Burrow coffee table and TV stand", suspected_fraud: false'
    },
    {
        "example": 'product_id: 22, category: Furniture, brand: Maiden Home, condition: New, price: 2200, seller_reputation: 4, description: "Brand new Maiden Home velvet sectional sofa", suspected_fraud: false'
    },
    {
        "example": 'product_id: 23, category: Furniture, brand: Castlety, condition: Used, price: 1300, seller_reputation: 3, description: "Used Castlety king-size bed frame", suspected_fraud: false'
    },
    {
        "example": 'product_id: 24, category: Furniture, brand: FLOYD, condition: New, price: 950, seller_reputation: 4, description: "Brand new FLOYD bed frame with storage", suspected_fraud: false'
    },
    {
        "example": 'product_id: 25, category: Furniture, brand: Benchmade Modern, condition: New, price: 2100, seller_reputation: 5, description: "Custom Benchmade Modern sectional", suspected_fraud: true'
    },
]

In [4]:
# Define a template for your examples
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

# Define your few-shot prompt template
prompt_template = FewShotPromptTemplate(
    prefix="Generate synthetic product information based on the following examples:",
    examples=examples,  # Your provided examples go here
    example_prompt=OPENAI_TEMPLATE,  # The example structure
    suffix="Generate a synthetic product with the following attributes: product_id, category, brand, condition, price, seller_reputation, description, suspected_fraud.",
    input_variables=[
        "product_id",
        "category",
        "brand",
        "condition",
        "price",
        "seller_reputation",
        "description",
        "suspected_fraud",
    ],  # Inputs for the new product generation
)

# Now you can use this template for generating synthetic product data
generated_prompt = prompt_template.format(
    product_id="26",
    category="Electronics",
    brand="Samsung",
    condition="New",
    price="1200",
    seller_reputation="5",
    description="Brand new Samsung Galaxy S21",
    suspected_fraud="false",
)

print(generated_prompt)

Generate synthetic product information based on the following examples:

product_id: 1, category: Electronics, brand: Sony, condition: New, price: 950, seller_reputation: 4, description: "Brand new Sony Xperia 1", suspected_fraud: false

product_id: 2, category: Electronics, brand: Xiaomi, condition: Used, price: 300, seller_reputation: 2, description: "Used Xiaomi Mi 9", suspected_fraud: false

product_id: 3, category: Electronics, brand: OnePlus, condition: New, price: 700, seller_reputation: 5, description: "Brand new OnePlus 9 Pro", suspected_fraud: false

product_id: 4, category: Electronics, brand: Huawei, condition: Used, price: 400, seller_reputation: 3, description: "Used Huawei P30", suspected_fraud: false

product_id: 5, category: Electronics, brand: Google, condition: New, price: 850, seller_reputation: 5, description: "Brand new Google Pixel 5", suspected_fraud: false

product_id: 6, category: Electronics, brand: Motorola, condition: Used, price: 250, seller_reputation: 4,

In [48]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

In [50]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=ProductListing,
    llm=ChatOpenAI(temperature=1),
    prompt=prompt_template,
)

/opt/anaconda3/envs/synthetic/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `ProductListing` to V2.
  obj = ForwardRef(obj)


PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.PlainValidatorFunctionSchema ({'type': 'with-info', 'function': <bound method BaseModel.validate of <class '__main__.ProductListing'>>})

For further information visit https://errors.pydantic.dev/2.10/u/invalid-for-json-schema

In [49]:
synthetic_results = synthetic_data_generator.generate(
    subject="medical_billing",
    extra="the name must be chosen at random. Make it something you wouldn't normally choose.",
    runs=10,
)

NameError: name 'synthetic_data_generator' is not defined